### 1.Importing modules and packages

In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [4]:
train_path = 'Datasets/train'
valid_path = 'Datasets/test'

### 2.Import the Transfer Learning Model

In [5]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

###### [3] meant for three channels, Red, Green, Blue

###### include_top=False=> to remove last layers as i am going to use categories as last layer

In [6]:
# So don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

##### Since you have already trained your model, not exactly. You are using VGG16 which already has trained weights, so do not train the models as shown Above

In [7]:
# useful for getting number of classes
folders = glob('Datasets/train/*')

In [8]:
folders

['Datasets/train\\NORMAL', 'Datasets/train\\PNEUMONIA']

In [9]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [10]:
x

<tf.Tensor 'flatten_1/Reshape:0' shape=(None, None) dtype=float32>

In [11]:
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

### Initializing  the Model

In [12]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [13]:
# view the structure of the model
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

#### ImageDataGenerator needed to create as many images to increase the dataset size by sgearing, zooming in and flipping left to right

###### Rescaling both test and train are necessary

In [15]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [16]:
training_set = train_datagen.flow_from_directory('Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')#'binary' also can be used in place of 'categorical'

Found 5216 images belonging to 2 classes.


###### training_set is simply a directory so is the test_set

In [19]:
test_set = test_datagen.flow_from_directory('Datasets/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 624 images belonging to 2 classes.


###### Lets use the Transfer Lerning technique over our image folders

In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
151/163 [==========================>...] - ETA: 3:29 - loss: 0.2214 - accuracy: 0.9158

###### One thing which is most important is Validation accuracy and 
Validation accuracy being closer to the training accuracy

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')



In [ ]:
# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')



In [ ]:
import tensorflow as tf

from keras.models import load_model

model.save('model_vgg19.h5')

